In [ ]:
!pip install

In [ ]:
import os
import sys
import logging

import numpy as np
import pandas as pd
from sagemaker.local import LocalSession
import sagemaker
from sagemaker.pytorch import PyTorch
import torch

In [ ]:
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
train_input_path = f"s3://{bucket}/imdb/data/small/train.csv"
test_input_path = f"s3://{bucket}/imdb/data/small/test.csv"

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")

## Training

In [ ]:
from sagemaker.huggingface import HuggingFace


# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'per_device_train_batch_size': 32,
                 'model_name': 'distilbert-base-uncased'
                 }

# create the Estimator
estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./source',
        instance_type='ml.g4dn.xlarge',  # Note: needs to be an instance with gpu
        instance_count=1,
        role=role,
        transformers_version='4.4',
        pytorch_version='1.6',
        py_version='py36',
        hyperparameters = hyperparameters
)
estimator.fit({'train': train_input_path, 'test': test_input_path})

## Inference

In [ ]:
# model_data = estimator.model_data
model_data = 's3://sagemaker-eu-central-1-611215368770/huggingface-pytorch-training-2021-06-01-18-10-34-763/output/model.tar.gz'
print(model_data)

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(
    model_data="s3://sagemaker-eu-central-1-611215368770/pytorch-training-2021-06-01-09-20-56-523/model.tar.gz",
    role=role,
    framework_version="1.7.1",
    source_dir="source",
    py_version="py3",
    entry_point="inference.py",
)


### Endpoint

In [ ]:
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.StringDeserializer(),
)

In [ ]:
input_1 = "this is a very good movie"
predictor.predict(input_1)

In [ ]:
input_2 = ['this movie sucks', 'this movie is ok']
predictor.predict(input_2)

In [ ]:
input_3 = ['such a terrible movie', 'what a great movie', 'omg best movie ever']
predictor.predict(input_3)

In [ ]:
predictor.delete_endpoint()

### Batch Transform

In [ ]:
# Just a 200-line csv file needs "ml.m5.2xlarge"? Crazy
transformer = pytorch_model.transformer(instance_count=1, instance_type="ml.m5.large", accept='text/csv', strategy ='SingleRecord')

In [ ]:
transformer.transform(test_input_path, content_type='text/csv', split_type='Line')

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./

In [ ]:
res = pd.read_csv("test.csv.out", header=None)
res

### Local

In [ ]:
%%sh -s $model_data
rm -rf model
mkdir model
aws s3 cp $1 model/ 
tar xvzf model/model.tar.gz --directory ./model

In [ ]:
!pip install "sagemaker>=2.31.0" "transformers==4.4.2" "datasets[s3]==1.5.0" --upgrade

In [ ]:
# pytorch_model.bin stores fine-tuned huggingface
# tokenizer_config.json stores tokenizer used for training

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig

model = AutoModelForSequenceClassification.from_pretrained('model')
tokenizer = AutoTokenizer.from_pretrained('model')

In [ ]:
inputs = ['this is a terrific movie']

In [ ]:
tokenized_input = tokenizer(inputs, truncation=True, padding=True)

In [ ]:
input_ids = torch.Tensor(tokenized_input['input_ids']).long()
attention_mask = torch.Tensor(tokenized_input['attention_mask']).long()

In [ ]:
model.eval()
with torch.no_grad():
    logits = model(input_ids, attention_mask).logits
    res = np.argmax(logits)
    print('logits:', logits, 'neg/pos:', res)